In [1]:
import sys
import pandas as pd
import os
from path import Path
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.request import urlopen
import json
import statsmodels as sm
from statsmodels.formula.api import ols
from Bio import Seq, SeqIO, AlignIO, Phylo, Align
from jinja2 import Environment, FileSystemLoader  # html template engine
import cv2
import numpy as np
import skimage as sk
import matplotlib.pylab as plt
import datetime as dt

In [2]:
sys.path.append('../')

In [3]:
import bjorn_support as bs
import onion_trees as ot
import mutations as bm
import visualize as bv
import reports as br

In [4]:
msa_fp = '/home/al/analysis/gisaid/sequences_2021-01-20_16-44_aligned.fasta.gz'
meta_fp = '/home/al/analysis/gisaid/metadata_2021-01-20_17-49.tsv.gz'

In [21]:
cols = {'Virus name': 'strain', 'Collection date': 'date' , 'Additional host information': 'purpose_of_sequencing',
        'Lineage': 'pangolin_lineage', 'Host': 'host'}
meta_fp = '/home/al/analysis/gisaid/metadata_2021-01-21.tsv.gz'
old_meta_fp = '/home/al/analysis/gisaid/metadata_2021-01-20_17-49.tsv.gz'
xtra_fp = '/home/al/analysis/gisaid/gisaid_hcov-19_2021_01_21_15.tsv'
drop_cols = ['Accession ID', 'Location', 'Passage', 'Specimen', 'Sequencing technology',
             'Assembly method', 'Comment', 'Comment type']

In [22]:
meta = bs.integrate_gisaid_meta(old_meta_fp, xtra_fp, msa_fp, cols, drop_cols)

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning:

Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.



In [23]:
meta_fp

'/home/al/analysis/gisaid/metadata_2021-01-21.tsv.gz'

In [29]:
# meta.to_csv(meta_fp, sep='\t', compression='gzip', index=False)

In [25]:
# meta = pd.read_csv(meta_fp, sep='\t', compression='gzip')
# meta.loc[(meta['country'].str.contains('United States'))&(meta['pangolin_lineage']=='B.1.1.7')]

In [26]:
# gisaid, _ = bm.identify_replacements_per_sample(msa_fp, meta_fp, bm.GENE2POS, data_src='gisaid', test=False)
# gisaid['mutation'] = gisaid['gene'] + ':' + gisaid['ref_aa'] + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
# gisaid.loc[gisaid['country']=='USA', 'country'] = 'United States of America'

In [27]:
# gisaid_fp = '/home/al/analysis/gisaid/subs_long_2021-01-15_14-55v2.csv.gz'
# gisaid.to_csv(gisaid_fp, index=False, compression='gzip')

In [28]:
# gisaid[(gisaid['mutation']=='ORF1b:D1183Y')]['strain'].unique().shape

In [5]:
# feature = 'mutations'
# values = ['S:S13I', 'S:W152C', 'S:L452R']
feature = 'pangolin_lineage'
# S:Q677H, M:A85S, N:D377Y,
values = ['B.1.1.7']#, 'S:W152C', 'S:L452R'] # 'ORF1ab:I4205V', 
input_params = {
    'gisaid_data_fp' : '/home/al/analysis/gisaid/subs_long_2021-01-20.csv.gz',
    'gisaid_meta_fp': '/home/al/analysis/gisaid/metadata_2021-01-21.tsv.gz',
    'vocs': ['B.1.1.7', 'B.1.1.70'],
    'strain': 'B117',
    'date': '01/21/2021',
    'msa_fp': Path('/home/al/analysis/gisaid/sequences_2021-01-20_16-44_aligned.fasta.gz'),
    'meta_fp' : Path('/home/al/code/HCoV-19-Genomics/metadata.csv'),
    'tree_fp' : Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile'),
    'subs_fp' : '/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv',
    'countries_fp' : '/home/al/data/geojsons/countries.geo.json',
    'states_fp' : "/home/al/data/geojsons/us-states.json",
    'counties_fp' : '/home/al/data/geojsons/us-counties.json',
    'patient_zero' : 'NC_045512.2',
    'b117_meta' : '/home/al/analysis/b117/nextstrain_groups_neherlab_ncov_S.N501_metadata.tsv',
    'b117_tree': 'test_data/b117_seqs_aligned.fasta.treefile',
    'sample_sz': 500,
    'sampling_img_fp' : "/home/al/analysis/b117/figs/sars-cov-2_EM_v3.png"
}

In [6]:
results = br.generate_voc_data(feature, values, input_params)

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (8,28) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
html = br.generate_voc_html(feature, values, results, template_name='voc.html')
# br.save_html(html, f'test_data/orf1ab_i4205v_report.html')
br.save_html(html, f'test_data/b117_report.html')

In [ ]:
plotly.offline.plot()

In [17]:
gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [16]:
# res = (gisaid.groupby(['date', 'country', 'division', 
#                                         'purpose_of_sequencing',
#                                         'location', 'pangolin_lineage', 'strain'])
#                        .agg(mutations=('mutation', 'unique')).reset_index())
# res['is_vui'] = res['mutations'].apply(bv.is_vui, args=(set(values),))

In [21]:
df

,date,num_samples,county_counts,counties,total_samples,cum_num_samples,cum_total_samples,rel_freq
0,2020-05-11,1,[1],[Alameda County],1,1,1,1.0
1,2020-07-06,1,[1],[unk],1,2,2,1.0
2,2020-07-13,2,[2],[unk],2,4,4,1.0
3,2020-09-21,1,[1],[San Joaquin County],1,5,5,1.0
4,2020-10-05,1,[1],[Orange County CA],1,6,6,1.0
5,2020-10-12,2,[2],[Orange County CA],2,8,8,1.0
6,2020-10-26,1,[1],[San Diego],1,9,9,1.0
7,2020-11-02,9,"[6, 3]","[Orange County CA, San Luis Obispo County]",9,18,18,1.0
8,2020-11-09,23,"[1, 5, 9, 6, 2]","[Humboldt County, Los Angeles County, Orange C...",23,41,41,1.0
9,2020-11-16,27,"[8, 15, 1, 3]","[Los Angeles County, Orange County CA, San Die...",27,68,68,1.0


In [18]:
gisaid['tmp'] = gisaid['date'].str.split('-')
gisaid = gisaid[gisaid['tmp'].str.len()>=3]
gisaid['date'] = pd.to_datetime(gisaid['date'], errors='coerce')

In [47]:
gisaid.loc[(gisaid['pos']>=13468)&(gisaid['pos']<=21555)
           &(gisaid['ref_aa']=='D')&(gisaid['alt_aa']=='Y')
           &(gisaid['division']=='California')]#.value_counts()

,idx,replacements,pos,gene,codon_num,ref_codon,alt_codon,ref_aa,alt_aa,strain,...,GISAID_clade,originating_lab,submitting_lab,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,mutation


In [48]:
sois = gisaid.loc[(gisaid['mutation']=='S:L452R')&(gisaid['division']=='California'), 'strain'].unique()
gisaid.loc[gisaid['strain'].isin(sois)
          &(gisaid['ref_aa']=='D')&(gisaid['alt_aa']=='Y')]

,idx,replacements,pos,gene,codon_num,ref_codon,alt_codon,ref_aa,alt_aa,strain,...,GISAID_clade,originating_lab,submitting_lab,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,mutation
3808399,USA/CA-ALSR-5104/2020,27357:t,27357,ORF6,53,GAT,TAT,D,Y,USA/CA-ALSR-5104/2020,...,GH,San Diego County Public Health Laboratory,Andersen lab at Scripps Research,SEARCH Alliance San Diego with Tracy Basler et al,https://www.gisaid.org,?,?,2020-12-23,?,ORF6:D53Y
3983750,USA/CA-CZB-15979/2020,29473:t,29473,N,401,GAT,TAT,D,Y,USA/CA-CZB-15979/2020,...,GH,Santa Clara County Public Health Laboratory,Chan-Zuckerberg Biohub,CZB Cliahub Consortium et al,https://www.gisaid.org,?,?,2020-12-29,?,N:D401Y
4254693,USA/CA-CHLA-PLM78606325/2020,27146:t,27146,M,209,GAC,TAC,D,Y,USA/CA-CHLA-PLM78606325/2020,...,GH,"Clinical virology Laboratory, Children's Hospi...","Center for Personalized Medicine, Children's H...",Gai et al et al,https://www.gisaid.org,?,?,2020-12-31,?,M:D209Y
4256209,USA/CA-CHLA-PLM08108247/2020,24940:t,24940,S,1127,GAT,TAT,D,Y,USA/CA-CHLA-PLM08108247/2020,...,GH,"Clinical virology Laboratory, Children's Hospi...","Center for Personalized Medicine, Children's H...",Gai et al et al,https://www.gisaid.org,?,?,2020-12-31,?,S:D1127Y
4256588,USA/CA-CHLA-PLM66857248/2020,29473:t,29473,N,401,GAT,TAT,D,Y,USA/CA-CHLA-PLM66857248/2020,...,G,"Clinical virology Laboratory, Children's Hospi...","Center for Personalized Medicine, Children's H...",Gai et al et al,https://www.gisaid.org,?,?,2020-12-31,?,N:D401Y
4968537,USA/CA-LACPHL-AF00109/2021,29473:t,29473,N,401,GAT,TAT,D,Y,USA/CA-LACPHL-AF00109/2021,...,G,USC Clinical Lab,Los Angeles County PHL,P. Hemarajata et al. et al,https://www.gisaid.org,?,?,2021-01-11,?,N:D401Y
5262929,USA/CA-SC20028/2020,29473:t,29473,N,401,GAT,TAT,D,Y,USA/CA-SC20028/2020,...,GH,Santa Clara County Public Health Laboratory,Santa Clara County Public Health Laboratory,Santa Clara County Public Health Department et al,https://www.gisaid.org,?,?,2021-01-13,?,N:D401Y
5521733,USA/CA-CSMC219/2020,29473:t,29473,N,401,GAT,TAT,D,Y,USA/CA-CSMC219/2020,...,G,"Cedars-Sinai Medical Center, Department of Pat...","Cedars-Sinai Medical Center, Molecular Patholo...",Wenjuan Zhang et al,https://www.gisaid.org,?,?,2021-01-14,?,N:D401Y
5522759,USA/CA-CSMC269/2020,29473:t,29473,N,401,GAT,TAT,D,Y,USA/CA-CSMC269/2020,...,G,"Cedars-Sinai Medical Center, Department of Pat...","Cedars-Sinai Medical Center, Molecular Patholo...",Wenjuan Zhang et al,https://www.gisaid.org,?,?,2021-01-14,?,N:D401Y


In [109]:
def world_time_relative(data, feature, values, res, strain='B117', vocs=['B.1.1.7', 'B.1.1.70']):
    if len(values)==1:
        results = (data.loc[(data[feature]==values[0])]
                            .drop_duplicates(subset=['date', 'strain']))
    else:
        # results = (data.groupby(['date', 'country', 'division', 'purpose_of_sequencing',
        #                          'location', 'pangolin_lineage', 'strain'])
        #                .agg(mutations=('mutation', 'unique')).reset_index())
        # results['is_vui'] = results['mutations'].apply(is_vui, args=(set(values),))
        res = res.copy()
        res.loc[:, 'tmp'] = res['date'].str.split('-')
        res = res[res['tmp'].str.len()>=3]
        res.loc[:, 'date'] = pd.to_datetime(res['date'], errors='coerce')
        res.loc[:, 'weekday'] = res['date'].dt.weekday
        res.loc[:, 'date'] = res['date'] - res['weekday'] * dt.timedelta(days=1)
        total_samples = (res[(~res['pangolin_lineage'].isin(vocs))]
                         .groupby('date')
                         .agg(total_samples=('strain', 'nunique'))
                         .reset_index())
        results = res[(res['is_vui']==True)].drop_duplicates(subset=['date', 'strain'])
        
    b117_world_time = (results.groupby('date')
                              .agg(num_samples=('strain', 'nunique'),
                                   country_counts=('country', 
                                                    lambda x: np.unique(x, 
                                                                        return_counts=True)),
                                   divisions=('division', 'unique'),
                                   locations=('location', 'unique'))
                              .reset_index())
    b117_world_time.loc[:, 'countries'] = b117_world_time['country_counts'].apply(lambda x: list(x[0]))
    b117_world_time.loc[:, 'country_counts'] = b117_world_time['country_counts'].apply(lambda x: list(x[1]))
    b117_world_time = pd.merge(b117_world_time, total_samples, on='date', how='right')
    b117_world_time.loc[:, ['num_samples', 'total_samples']] = b117_world_time[['num_samples', 'total_samples']].fillna(0)
    first_detected = b117_world_time.loc[b117_world_time['num_samples']>0]['date'].min()
    first_countries = b117_world_time.loc[b117_world_time['date']==first_detected, 'countries'].values[0]
    b117_world_time = b117_world_time[b117_world_time['date']>=first_detected]
    b117_world_time['cum_num_samples'] = b117_world_time['num_samples'].cumsum()
    b117_world_time.loc[:, 'cum_total_samples'] = b117_world_time['total_samples'].cumsum()
    b117_world_time.loc[:, 'rel_freq'] = b117_world_time['cum_num_samples'] / b117_world_time['cum_total_samples']
    fig = go.Figure(data=go.Scatter(y=b117_world_time['rel_freq'], 
                                    x=b117_world_time['date'], 
                                    name='B.1.1.7 samples', mode='markers+lines', 
                                    line_color='rgba(220,20,60,.6)',
                                    text=b117_world_time[['num_samples', 'countries', 'country_counts',
                                                          'divisions', 'locations', 
                                                          'date']],
                                    hovertemplate="<b>Number of cases: %{text[0]}</b><br>" +
                                                  "<b>Country(s) Reported: %{text[1]}</b><br>" +
                                                  "<b>Cases Per Country: %{text[2]}</b><br>" +
                                                  "<b>State(s) Reported: %{text[3]}</b><br>" +
                                                  "<b>County(s) Reported: %{text[4]}</b><br>" +
                                                  "<b>Date: %{text[5]}</b><br>"))
    fig.add_annotation(x=first_detected, 
                       y=b117_world_time.loc[b117_world_time['date']==first_detected, 'rel_freq'].values[0],
            text=f"On Earth, {strain} 1st detected in <br> {', '.join(first_countries)} <br> on week of <br> {first_detected.date()}",
            showarrow=True,
            arrowhead=1, yshift=10, arrowsize=2, ay=-250, ax=100)
    fig.update_layout(yaxis_title=f'Relative cumulative number of {strain} cases on Earth', 
                      xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850,
    fig.update_yaxes(side = 'right')
    return fig

fig = world_time_relative(gisaid, feature, values, res.copy(), strain=input_params['strain'])
fig.show()

In [101]:
df

,date,num_samples,country_counts,divisions,locations,countries,total_samples,cum_num_samples,cum_total_samples,rel_freq
29,2020-07-13,1.0,[1],[California],[unk],[United States of America],4858,1.0,4858,0.000206
30,2020-07-20,0.0,NaN,NaN,NaN,NaN,4541,1.0,9399,0.000106
31,2020-07-27,0.0,NaN,NaN,NaN,NaN,4675,1.0,14074,0.000071
32,2020-08-03,0.0,NaN,NaN,NaN,NaN,4547,1.0,18621,0.000054
33,2020-08-10,0.0,NaN,NaN,NaN,NaN,5249,1.0,23870,0.000042
34,2020-08-17,0.0,NaN,NaN,NaN,NaN,5379,1.0,29249,0.000034
35,2020-08-24,0.0,NaN,NaN,NaN,NaN,5319,1.0,34568,0.000029
36,2020-08-31,0.0,NaN,NaN,NaN,NaN,5556,1.0,40124,0.000025
37,2020-09-07,0.0,NaN,NaN,NaN,NaN,5729,1.0,45853,0.000022
38,2020-09-14,0.0,NaN,NaN,NaN,NaN,6586,1.0,52439,0.000019


In [ ]:
## TODO: look at col name with clade (e.g. 20C) values to use in mutation_diversity plots

In [21]:
# results['ca_time']

In [16]:
gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [12]:

def get_mutations(data: pd.DataFrame, lineage: str='B.1.1.7'):
    mutations = set(data[data['pangolin_lineage']==lineage]['mutation'].unique().tolist())
    return mutations

In [12]:
# gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')
# gisaid.drop(columns=['mutation'], inplace=True)
# gisaid['mutations'] = gisaid['gene'] + ':' + gisaid['ref_aa'] + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
# gisaid.to_csv(input_params['gisaid_data_fp'], compression='gzip')

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38,39,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.7']['strain'].unique().shape

(10421,)

In [21]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.70']['strain'].unique().shape

(1340,)

In [18]:
b117 = get_mutations(gisaid, lineage='B.1.1.7')
b1170 = get_mutations(gisaid, lineage='B.1.1.70')
b117.intersection(b1170)

{'3UTR:A15T',
 '3UTR:G35G',
 '3UTR:I4I',
 '3UTR:Q66K',
 '3UTR:T32I',
 '3UTR:V46L',
 '3UTR:V6L',
 '3UTR:_31L',
 '3UTR:_9W',
 '5UTR:I74I',
 '5UTR:P73P',
 '5UTR:R81C',
 '5UTR:S68L',
 'E:F23F',
 'E:L65L',
 'E:L73F',
 'E:S68F',
 'E:T30I',
 'M:D160D',
 'M:D190D',
 'M:F112F',
 'M:F53F',
 'M:G202G',
 'M:G78C',
 'M:L124L',
 'M:L35L',
 'M:N203N',
 'M:N41N',
 'M:S173K',
 'M:T172T',
 'M:Y196Y',
 'M:Y71Y',
 'N:A134V',
 'N:A156S',
 'N:A208S',
 'N:A211S',
 'N:A252S',
 'N:A273S',
 'N:A359S',
 'N:A35A',
 'N:A90S',
 'N:D348Y',
 'N:D401Y',
 'N:D402Y',
 'N:D415D',
 'N:F110F',
 'N:F274F',
 'N:F346F',
 'N:G204R',
 'N:H145H',
 'N:I292I',
 'N:I337I',
 'N:N126N',
 'N:N192N',
 'N:P142S',
 'N:P199S',
 'N:P383P',
 'N:Q28H',
 'N:Q418H',
 'N:Q9H',
 'N:R195I',
 'N:R203K',
 'N:R385I',
 'N:S194P',
 'N:S413I',
 'N:T135I',
 'N:T16M',
 'N:T16T',
 'N:T24I',
 'N:T282T',
 'N:T379I',
 'N:T417I',
 'N:T54T',
 'N:Y298Y',
 'N:Y360Y',
 'N:Y86Y',
 'ORF10:C19C',
 'ORF10:Y26Y',
 'ORF1ab:A1151V',
 'ORF1ab:A1283V',
 'ORF1ab:A1298V',
 

In [13]:
# gisaid.columns

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (37,38,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.



'England/205141731/2020', 'Lebanon/LAU-uk4/2020', 'England/204590625/2020', 
'England/204590575/2020', 'England/NORW-F6D33/2020'

In [16]:
g = gisaid.groupby(['pangolin_lineage', 'strain']).agg(mutations=('mutation', 'unique')).reset_index()
def check_mutation(x, mutation='S:N501Y'):
    if mutation in x:
        return True
    return False

g['501Y'] = g['mutations'].apply(check_mutation)

In [31]:
m = "S:D614G"
g['614G'] = g['mutations'].apply(check_mutation, args=(m,))

In [32]:
soi = g.loc[(g['614G']==False) 
          & (g['pangolin_lineage']=='B.1.1.7'), 
            'strain'].unique().tolist() + [input_params['patient_zero']]

In [33]:
soi

['England/CAMC-CFDB18/2020', 'NC_045512.2']

In [26]:
bs.fetch_seqs(input_params['msa_fp'], 'test_data/b117_wo_N501Y.fasta', soi, is_aligned=True)

1

In [185]:
# gisaid[(gisaid['pangolin_lineage']=='B.1.1.7')]['location'].value_counts()

In [186]:
# df

In [187]:
results['county_map'] = fig

In [143]:
gisaid.loc[(gisaid['codon_num']==452) & (gisaid['gene']=='S') & (gisaid['alt_aa']=='R'), 'strain'].unique().shape

(404,)

In [24]:
gisaid.loc[(gisaid['codon_num']==452) & (gisaid['gene']=='S') & (gisaid['alt_aa']=='R'), 'date'].min()

'2020-03-17'

In [ ]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.70']

In [15]:
dists.loc[(dists['group']=='outgroup') & (dists['genetic_distance']>0.0008), 'pangolin_lineage'].value_counts()

B.1.1.70     48
B.1.351      19
B.1.160       1
B.1.1.31      1
B.1.160.6     1
B.1.1.1       1
B.1.367       1
C.14          1
Name: pangolin_lineage, dtype: int64